In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib agg


import global_settings
from utils import settings, experiments, results
import jax
from multiprocessing import Pool
from tqdm import tqdm
import jax.numpy as jnp
import os

In [2]:
experiment = experiments.ExperimentSampleConstrained(
    settings=settings.SettingsExperimentSample(
        output_path=global_settings.PATH_RESULTS_FIXED,
        dataset="regression2d",
        dataset_normalization="standardization",
        hidden_layers=1,
        hidden_neurons=3,
        activation="tanh",
        activation_last_layer="none",
        num_warmup=2**10,
        statistic="reduced",
        statistic_p=0.99,
        samples_per_chain=1,
        identifiable_modes=3,
        pool_size=8,
        seed=0
    )
)

normalized probabilities: [0.5665243  0.3504857  0.08298998]


In [3]:
def run_parallel(key):
    experiment.sample(key)
    return experiment._mcmc.get_samples()

rng_key, *sub_keys = jax.random.split(jax.random.PRNGKey(experiment._settings.seed), experiment._sample_statistics["num_chains"] + 1)
with Pool(experiment._settings.pool_size) as p:
    samples_parallel = list(tqdm(p.imap(run_parallel, sub_keys), total=experiment._sample_statistics["num_chains"]))

samples = {}
for samples_run in samples_parallel:
    for key in samples_run.keys():
        if key not in samples:
            samples[key] = samples_run[key]
        else:
            samples[key] = jnp.concatenate([samples[key], samples_run[key]])
experiment._samples = samples

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1274/1274 [08:29<00:00,  2.50it/s]


In [4]:
print(f"saved as {experiment.save()}")

saved as /home/gw/data/experiments/master_thesis/results_fixed/25a6723e6c442799e19c4876bdd06f4b.tar.gz
